# Recommendation Systems
# Neural Collaborative Filtering

## Dataset Configurations from GitHub

For loading dataset, we use the dataset from the GitHub [repo](https://github.com/hexiangnan/neural_collaborative_filtering) of the original paper. The datasets in the research are:
1. Movielens-1M
2. Pinterest

Detailed information is availaible in the paper itself.

In [1]:
!git clone https://github.com/hexiangnan/neural_collaborative_filtering.git

Cloning into 'neural_collaborative_filtering'...


In [2]:
# !cp '/kaggle/working/neural_collaborative_filtering/Data' '/kaggle/working/' -r

import os
print(os.getcwd())
print(os.listdir(os.getcwd()))

c:\Users\dhruv\Documents\DA-IICT\Arpit_rana\MajorProject\previous_code
['movielens.ipynb', 'ncf_tensorflow-main.zip', 'neural_collaborative_filtering']


In [3]:
os.chdir('neural_collaborative_filtering')
print(os.getcwd())
print(os.listdir(os.getcwd()))

c:\Users\dhruv\Documents\DA-IICT\Arpit_rana\MajorProject\previous_code\neural_collaborative_filtering
['.git', 'Data', 'Dataset.py', 'Dockerfile', 'evaluate.py', 'GMF.py', 'LICENSE', 'MLP.py', 'NeuMF.py', 'Pretrain', 'README.md']


# Tensorflow Version

In [4]:
import warnings
warnings.filterwarnings('ignore')
import logging
logging.basicConfig(level=logging.ERROR) 

import numpy as np
import scipy.sparse as sp
from time import time
import pandas as pd

In [5]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input, Model
from tensorflow.keras import initializers, regularizers
from tensorflow.keras.layers import Embedding, Dense, Flatten, concatenate, multiply
from tensorflow.keras.optimizers import Adam, Adagrad, RMSprop, SGD

## Dataset

In [7]:
class Dataset(object):
    '''
    Dataset Class for making dataset input for the models
    trainMatrix: training Matrix of the data
    testRatings: positive test interactions
    testNegatives: negative test interactions sampled for each user
    '''

    def __init__(self, path):
        '''
        Constructor
        '''
        self.trainMatrix = self.load_rating_file_as_matrix(path + ".train.rating")
        self.testRatings = self.load_rating_file_as_list(path + ".test.rating")
        self.testNegatives = self.load_negative_file(path + ".test.negative")
        assert len(self.testRatings) == len(self.testNegatives)

        self.num_users, self.num_items = self.trainMatrix.shape

    def load_rating_file_as_list(self, filename):
        ratingList = []
        with open(filename, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split("\t")
                user, item = int(arr[0]), int(arr[1])
                ratingList.append([user, item])
                line = f.readline()
        return ratingList

    def load_negative_file(self, filename):
        negativeList = []
        with open(filename, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split("\t")
                negatives = []
                for x in arr[1: ]:
                    negatives.append(int(x))
                negativeList.append(negatives)
                line = f.readline()
        return negativeList

    def load_rating_file_as_matrix(self, filename):
        '''
        Read .rating file and Return dok matrix.
        The first line of .rating file is: num_users\t num_items
        '''
        # Get number of users and items
        num_users, num_items = 0, 0
        with open(filename, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split("\t")
                u, i = int(arr[0]), int(arr[1])
                num_users = max(num_users, u)
                num_items = max(num_items, i)
                line = f.readline()
        # Construct matrix
        mat = sp.dok_matrix((num_users+1, num_items+1), dtype=np.float32)
        with open(filename, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split("\t")
                user, item, rating = int(arr[0]), int(arr[1]), float(arr[2])
                if (rating > 0):
                    mat[user, item] = 1.0
                line = f.readline()
        return mat

## Evaluation

In [32]:
import math
import heapq # for retrieval topK
import multiprocessing
import numpy as np
from time import time
#from numba import jit, autojit

# Global variables that are shared across processes
_model = None
_testRatings = None
_testNegatives = None
_K = None

def evaluate_model(model, testRatings, testNegatives, K, num_thread):
    
    """
    Evaluate the performance (Hit_Ratio, NDCG) of top-K recommendation
    Return: score of each test rating.
    """
    
    global _model
    global _testRatings
    global _testNegatives
    global _K
    _model = model
    _testRatings = testRatings
    _testNegatives = testNegatives
    _K = K

    hits, ndcgs = [],[]
    if(num_thread > 1): # Multi-thread
        pool = multiprocessing.Pool(processes=num_thread)
        res = pool.map(eval_one_rating, range(len(_testRatings)))
        pool.close()
        pool.join()
        hits = [r[0] for r in res]
        ndcgs = [r[1] for r in res]
        return (hits, ndcgs)
    # Single thread
    for idx in range(len(_testRatings)):
        (hr,ndcg) = eval_one_rating(idx)
        hits.append(hr)
        ndcgs.append(ndcg)
    return (hits, ndcgs)

def eval_one_rating(idx):
    rating = _testRatings[idx]
    items = _testNegatives[idx]
    u = rating[0]
    gtItem = rating[1]
    items.append(gtItem)
    # Get prediction scores
    map_item_score = {}
    users = np.full(len(items), u, dtype = 'int32')
    predictions = _model.predict([users, np.array(items)],
                                 batch_size=100, verbose=0)
    for i in range(len(items)):
        item = items[i]
        map_item_score[item] = predictions[i]
    items.pop()

    # Evaluate top rank list
    ranklist = heapq.nlargest(_K, map_item_score, key=map_item_score.get)
    hr = getHitRatio(ranklist, gtItem)
    ndcg = getNDCG(ranklist, gtItem)
    return (hr, ndcg)

def getHitRatio(ranklist, gtItem):
    for item in ranklist:
        if item == gtItem:
            return 1
    return 0

def getNDCG(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return math.log(2) / math.log(i+2)
    return 0

In [33]:
topK = 10
evaluation_threads = 1 #mp.cpu_count()

## GMF

In [34]:
print(os.getcwd())

c:\Users\dhruv\Documents\DA-IICT\Arpit_rana\MajorProject\previous_code\neural_collaborative_filtering


In [35]:
os.listdir(os.getcwd())

['.git',
 'Data',
 'Dataset.py',
 'Dockerfile',
 'evaluate.py',
 'GMF.py',
 'LICENSE',
 'MLP.py',
 'NeuMF.py',
 'Pretrain',
 'README.md']

In [36]:
from pathlib import Path 


Path(os.getcwd(), "Data/")

WindowsPath('c:/Users/dhruv/Documents/DA-IICT/Arpit_rana/MajorProject/previous_code/neural_collaborative_filtering/Data')

In [37]:
configurations = {
    'path': Path(os.getcwd()) / "Data",
    'dataset': 'ml-1m',
    'regs': [0, 0],
    'lr': 0.001,          ## Learning Rate
    'batch_size': 256,    ## Batch Size
    'epochs': 10,          ## Training Epochs
    'learner': 'adam',
    'num_factors': 10,
    'num_layers': 3,
    'num_neg': 2,
    'verbose': 2,
    'out': True,
}

print('Configurations: ')
for key, value in configurations.items():
  print(f'{key} : {value}')

Configurations: 
path : c:\Users\dhruv\Documents\DA-IICT\Arpit_rana\MajorProject\previous_code\neural_collaborative_filtering\Data
dataset : ml-1m
regs : [0, 0]
lr : 0.001
batch_size : 256
epochs : 10
learner : adam
num_factors : 10
num_layers : 3
num_neg : 2
verbose : 2
out : True


In [38]:
# Loading data
t1 = time()

dataset_path = os.path.join(configurations['path'], configurations['dataset'])
# print(dataset_path)

dataset = Dataset(dataset_path)

train, testRatings, testNegatives = dataset.trainMatrix, dataset.testRatings, dataset.testNegatives
num_users, num_items = train.shape
print("Load data done [%.1f s]. #user=%d, #item=%d, #train=%d, #test=%d"
      %(time()-t1, num_users, num_items, train.nnz, len(testRatings)))


Load data done [14.4 s]. #user=6040, #item=3706, #train=994169, #test=6040


In [39]:
def get_GMF_model(num_users, num_items, latent_dim, regs=[0,0]):
    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')

    MF_Embedding_User = Embedding(input_dim=num_users, output_dim=latent_dim, name='user_embedding',
                                    embeddings_initializer=initializers.RandomNormal(stddev=0.01),
                                    embeddings_regularizer=regularizers.l2(regs[0]))(user_input)
    MF_Embedding_Item = Embedding(input_dim=num_items, output_dim=latent_dim, name='item_embedding',
                                    embeddings_initializer=initializers.RandomNormal(stddev=0.01),
                                    embeddings_regularizer=regularizers.l2(regs[1]))(item_input)

    # Crucial to flatten an embedding vector!
    user_latent = Flatten()(MF_Embedding_User)
    item_latent = Flatten()(MF_Embedding_Item)

    # Element-wise product of user and item embeddings
    predict_vector = multiply([user_latent, item_latent])

    # Final prediction layer
    #prediction = Lambda(lambda x: K.sigmoid(K.sum(x)), output_shape=(1,))(predict_vector)
    prediction = Dense(1, activation='sigmoid', name = 'prediction')(predict_vector)

    model = Model(inputs=[user_input, item_input],
                outputs=prediction)

    return model

In [40]:
def get_train_instances(train, num_negatives):
    user_input, item_input, labels = [],[],[]
    num_users = train.shape[0]
    for (u, i) in train.keys():
        # positive instance
        user_input.append(u)
        item_input.append(i)
        labels.append(1)
        # negative instances
        for t in range(num_negatives):
            j = np.random.randint(num_items)
            while (u, j) in train:
                j = np.random.randint(num_items)
            user_input.append(u)
            item_input.append(j)
            labels.append(0)
    return user_input, item_input, labels

In [41]:
# Build model
model = get_GMF_model(num_users, num_items, configurations['num_factors'], configurations['regs'])
if configurations['learner'].lower() == "adagrad":
    model.compile(optimizer=Adagrad(learning_rate=configurations['lr']), loss='binary_crossentropy')
elif configurations['learner'].lower() == "rmsprop":
    model.compile(optimizer=RMSprop(learning_rate=configurations['lr']), loss='binary_crossentropy')
elif configurations['learner'].lower() == "adam":
    model.compile(optimizer=Adam(learning_rate=configurations['lr']), loss='binary_crossentropy')
else:
    model.compile(optimizer=SGD(learning_rate=configurations['lr']), loss='binary_crossentropy')
print(model.summary())


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_embedding      │ (None, 1, 10)     │     60,400 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_embedding      │ (None, 1, 10)     │     37,060 │ item_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 10)        │          0 │ user_embedding[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 10)        │          0 │ item_embedding[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 10)        │          0 │ flatten[0][0],    │
│                     │                   │            │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ prediction (Dense)  │ (None, 1)         │         11 │ multiply[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 97,471 (380.75 KB)

 Trainable params: 97,471 (380.75 KB)

 Non-trainable params: 0 (0.00 B)

None


In [42]:
topK = 10
evaluation_threads = 1 #mp.cpu_count()

In [43]:
# Init performance
t1 = time()
(hits, ndcgs) = evaluate_model(model, testRatings, testNegatives, topK, evaluation_threads)
hr, ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
#mf_embedding_norm = np.linalg.norm(model.get_layer('user_embedding').get_weights())+np.linalg.norm(model.get_layer('item_embedding').get_weights())
#p_norm = np.linalg.norm(model.get_layer('prediction').get_weights()[0])
print('Init: HR = %.4f, NDCG = %.4f\t [%.1f s]' % (hr, ndcg, time()-t1))

Init: HR = 0.1015, NDCG = 0.0465	 [457.4 s]


In [ ]:
# Train model

best_hr, best_ndcg, best_iter = hr, ndcg, -1

for epoch in range(configurations['epochs']):
    t1 = time()
    # Generate training instances
    user_input, item_input, labels = get_train_instances(train, configurations['num_neg'])

    # Training
    hist = model.fit([np.array(user_input), np.array(item_input)], #input
                      np.array(labels), # labels
                      batch_size=configurations['batch_size'], epochs=1, verbose=0, shuffle=True)
    t2 = time()

    # Evaluation
    if epoch % configurations['verbose'] == 0:
        
        (hits, ndcgs) = evaluate_model(model, testRatings, testNegatives, topK, evaluation_threads)
        
        hr, ndcg, loss = np.array(hits).mean(), np.array(ndcgs).mean(), hist.history['loss'][0]
        
        print('Iteration %d [%.1f s]: HR = %.4f, NDCG = %.4f, loss = %.4f [%.1f s]'
              % (epoch,  t2-t1, hr, ndcg, loss, time()-t2))
        
        if hr > best_hr:
        
            best_hr, best_ndcg, best_iter = hr, ndcg, epoch
            model_out_file = '%s_GMF_%d.weights.h5' %(configurations['dataset'], configurations['num_factors'])
        
            if configurations['out'] > 0:
                model.save_weights(model_out_file, overwrite=True)
        
            print("End. Best Iteration %d:  HR = %.4f, NDCG = %.4f. " %(best_iter, best_hr, best_ndcg))
            
if configurations['out'] > 0:
    
    print("The best GMF model is saved to %s" %(model_out_file))

## MLP

In [ ]:
configurations = {
    'path': Path(os.getcwd()) / "Data",
    'dataset': 'ml-1m',
    'reg_layers': [0, 0, 0],
    'lr': 0.001,          ## Learning Rate
    'batch_size': 256,    ## Batch Size
    'epochs': 10,          ## Training Epochs
    'learner': 'adam',
    'layers': [32, 16, 8],
    'num_neg': 2,
    'verbose': 2,
    'out': True,
}

print('Configurations: ')
for key, value in configurations.items():
  print(f'{key} : {value}')

Configurations: 
path : /kaggle/working/Data/
dataset : ml-1m
reg_layers : [0, 0, 0]
lr : 0.001
batch_size : 256
epochs : 10
learner : adam
layers : [32, 16, 8]
num_neg : 2
verbose : 2
out : True


In [18]:
def get_MLP_model(num_users, num_items, layers: list[int] = [20,10], reg_layers=[0,0]):
    assert len(layers) == len(reg_layers)

    num_layer = len(layers) #Number of layers in the MLP
    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')

    MF_Embedding_User = Embedding(input_dim=num_users, output_dim=layers[0]//2, name='user_embedding',
                                    embeddings_initializer=initializers.RandomNormal(stddev=0.01),
                                    embeddings_regularizer=regularizers.l2(reg_layers[0]))(user_input)
    MF_Embedding_Item = Embedding(input_dim=num_items, output_dim=layers[0]//2, name='item_embedding',
                                    embeddings_initializer=initializers.RandomNormal(stddev=0.01),
                                    embeddings_regularizer=regularizers.l2(reg_layers[1]))(item_input)

    # Crucial to flatten an embedding vector!
    user_latent = Flatten()(MF_Embedding_User)
    item_latent = Flatten()(MF_Embedding_Item)

    # The 0-th layer is the concatenation of embedding layers
    vector = concatenate([user_latent, item_latent])

    # MLP layers
    for idx in range(1, num_layer):
        layer = Dense(units=layers[idx], kernel_regularizer= regularizers.l2(reg_layers[idx]), activation='relu', name = 'layer%d' %idx)
        vector = layer(vector)

    # Final prediction layer
    prediction = Dense(units=1, activation='sigmoid', kernel_initializer='lecun_uniform', name = 'prediction')(vector)

    model = Model(inputs=[user_input, item_input],
                  outputs=prediction)

    return model

In [19]:
# Build model
model = get_MLP_model(num_users, num_items, configurations['layers'], configurations['reg_layers'])
if configurations['learner'].lower() == "adagrad":
    model.compile(optimizer=Adagrad(learning_rate=configurations['lr']), loss='binary_crossentropy')
elif configurations['learner'].lower() == "rmsprop":
    model.compile(optimizer=RMSprop(learning_rate=configurations['lr']), loss='binary_crossentropy')
elif configurations['learner'].lower() == "adam":
    model.compile(optimizer=Adam(learning_rate=configurations['lr']), loss='binary_crossentropy')
else:
    model.compile(optimizer=SGD(learning_rate=configurations['lr']), loss='binary_crossentropy')
print(model.summary())

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_embedding      │ (None, 1, 16)     │     96,640 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_embedding      │ (None, 1, 16)     │     59,296 │ item_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 16)        │          0 │ user_embedding[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_3 (Flatten) │ (None, 16)        │          0 │ item_embedding[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 32)        │          0 │ flatten_2[0][0],  │
│ (Concatenate)       │                   │            │ flatten_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer1 (Dense)      │ (None, 16)        │        528 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer2 (Dense)      │ (None, 8)         │        136 │ layer1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ prediction (Dense)  │ (None, 1)         │          9 │ layer2[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 156,609 (611.75 KB)

 Trainable params: 156,609 (611.75 KB)

 Non-trainable params: 0 (0.00 B)

None


In [20]:
topK = 10
evaluation_threads = 1 #mp.cpu_count()

In [21]:
# Init performance
t1 = time()
(hits, ndcgs) = evaluate_model(model, testRatings, testNegatives, topK, evaluation_threads)
hr, ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
#mf_embedding_norm = np.linalg.norm(model.get_layer('user_embedding').get_weights())+np.linalg.norm(model.get_layer('item_embedding').get_weights())
#p_norm = np.linalg.norm(model.get_layer('prediction').get_weights()[0])
print('Init: HR = %.4f, NDCG = %.4f\t [%.1f s]' % (hr, ndcg, time()-t1))

Init: HR = 0.0952, NDCG = 0.0373	 [331.3 s]


In [22]:
# Train model
best_hr, best_ndcg, best_iter = hr, ndcg, -1
for epoch in range(configurations['epochs']):
    t1 = time()
    # Generate training instances
    user_input, item_input, labels = get_train_instances(train, configurations['num_neg'])

    # Training
    hist = model.fit([np.array(user_input), np.array(item_input)], #input
                      np.array(labels), # labels
                      batch_size=configurations['batch_size'], epochs=1, verbose=0, shuffle=True)
    t2 = time()

    # Evaluation
    if epoch % configurations['verbose'] == 0:
        (hits, ndcgs) = evaluate_model(model, testRatings, testNegatives, topK, evaluation_threads)
        hr, ndcg, loss = np.array(hits).mean(), np.array(ndcgs).mean(), hist.history['loss'][0]
        print('Iteration %d [%.1f s]: HR = %.4f, NDCG = %.4f, loss = %.4f [%.1f s]'
              % (epoch,  t2-t1, hr, ndcg, loss, time()-t2))
        if hr > best_hr:
            best_hr, best_ndcg, best_iter = hr, ndcg, epoch
            model_out_file = '%s_MLP_%s.weights.h5' %(configurations['dataset'], configurations['layers'])
            if configurations['out'] > 0:
                model.save_weights(model_out_file, overwrite=True)
            print("End. Best Iteration %d:  HR = %.4f, NDCG = %.4f. " %(best_iter, best_hr, best_ndcg))
if configurations['out'] > 0:
    print("The best MLP model is saved to %s" %(model_out_file))

Iteration 0 [35.8 s]: HR = 0.4916, NDCG = 0.2658, loss = 0.4386 [330.8 s]
End. Best Iteration 0:  HR = 0.4916, NDCG = 0.2658. 
Iteration 2 [32.9 s]: HR = 0.5601, NDCG = 0.3100, loss = 0.3772 [333.1 s]
End. Best Iteration 2:  HR = 0.5601, NDCG = 0.3100. 
Iteration 4 [32.9 s]: HR = 0.5876, NDCG = 0.3293, loss = 0.3519 [334.9 s]
End. Best Iteration 4:  HR = 0.5876, NDCG = 0.3293. 
Iteration 6 [33.6 s]: HR = 0.6053, NDCG = 0.3414, loss = 0.3407 [335.8 s]
End. Best Iteration 6:  HR = 0.6053, NDCG = 0.3414. 
Iteration 8 [33.5 s]: HR = 0.6180, NDCG = 0.3513, loss = 0.3342 [359.2 s]
End. Best Iteration 8:  HR = 0.6180, NDCG = 0.3513. 
The best MLP model is saved to ml-1m_MLP_[32, 16, 8].weights.h5


## NeuMF

In [23]:
configurations = {
    'path': '/kaggle/working/Data/',
    'dataset': 'ml-1m',
    'epochs': 10,          ## Training Epochs
    'batch_size': 256,    ## Batch Size
    'num_factors': 10,
    'layers': [32, 16, 8],
    'reg_mf': 0,
    'reg_layers': [0, 0, 0],
    'num_neg': 2,
    'lr': 0.001,          ## Learning Rate
    'learner': 'adam',
    'verbose': 2,
    'out': True,
    'mf_pretrain': '',
    'mlp_pretrain': ''
}

print('Configurations: ')
for key, value in configurations.items():
  print(f'{key} : {value}')

Configurations: 
path : /kaggle/working/Data/
dataset : ml-1m
epochs : 10
batch_size : 256
num_factors : 10
layers : [32, 16, 8]
reg_mf : 0
reg_layers : [0, 0, 0]
num_neg : 2
lr : 0.001
learner : adam
verbose : 2
out : True
mf_pretrain : 
mlp_pretrain : 


In [24]:
def get_NeuMF_model(num_users, num_items, mf_dim=10, layers=[10], reg_layers=[0], reg_mf=0):
    assert len(layers) == len(reg_layers)
    num_layer = len(layers) #Number of layers in the MLP
    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = keras.Input(shape=(1,), dtype='int32', name = 'item_input')

    # Embedding layer
    MF_Embedding_User = Embedding(input_dim = num_users, output_dim = mf_dim, name = 'mf_embedding_user',
                                  embeddings_initializer=keras.initializers.RandomNormal(stddev=0.01), embeddings_regularizer = keras.regularizers.l2(reg_mf))(user_input)
    MF_Embedding_Item = Embedding(input_dim = num_items, output_dim = mf_dim, name = 'mf_embedding_item',
                                  embeddings_initializer=keras.initializers.RandomNormal(stddev=0.01), embeddings_regularizer = keras.regularizers.l2(reg_mf))(item_input)

    MLP_Embedding_User = Embedding(input_dim = num_users, output_dim = layers[0]//2, name = "mlp_embedding_user",
                                  embeddings_initializer=keras.initializers.RandomNormal(stddev=0.01), embeddings_regularizer = keras.regularizers.l2(reg_layers[0]))(user_input)
    MLP_Embedding_Item = Embedding(input_dim = num_items, output_dim = layers[0]//2, name = 'mlp_embedding_item',
                                  embeddings_initializer=keras.initializers.RandomNormal(stddev=0.01), embeddings_regularizer = keras.regularizers.l2(reg_layers[0]))(item_input)

    # MF part
    mf_user_latent = Flatten()(MF_Embedding_User)
    mf_item_latent = Flatten()(MF_Embedding_Item)
    mf_vector = multiply([mf_user_latent, mf_item_latent]) # element-wise multiply

    # MLP part
    mlp_user_latent = Flatten()(MLP_Embedding_User)
    mlp_item_latent = Flatten()(MLP_Embedding_Item)
    mlp_vector = concatenate([mlp_user_latent, mlp_item_latent])
    for idx in range(1, num_layer):
        layer = Dense(units=layers[idx], kernel_regularizer= keras.regularizers.l2(reg_layers[idx]), activation='relu', name="layer%d" %idx)
        mlp_vector = layer(mlp_vector)

    # Concatenate 
    #mf_vector = Lambda(lambda x: x * alpha)(mf_vector)
    #mlp_vector = Lambda(lambda x : x * (1-alpha))(mlp_vector)
        
    
    predict_vector = concatenate([mf_vector, mlp_vector])

    # Final prediction layer
    prediction = Dense(units=1, activation='sigmoid', kernel_initializer='lecun_uniform', name = "prediction")(predict_vector)

    model = Model(inputs=[user_input, item_input],
                  outputs=prediction)

    return model

In [25]:
# Build model
model = get_NeuMF_model(num_users, num_items, configurations['num_factors'], configurations['layers'], configurations['reg_layers'], configurations['reg_mf'])
if configurations['learner'].lower() == "adagrad":
    model.compile(optimizer=Adagrad(learning_rate=configurations['lr']), loss='binary_crossentropy')
elif configurations['learner'].lower() == "rmsprop":
    model.compile(optimizer=RMSprop(learning_rate=configurations['lr']), loss='binary_crossentropy')
elif configurations['learner'].lower() == "adam":
    model.compile(optimizer=Adam(learning_rate=configurations['lr']), loss='binary_crossentropy')
else:
    model.compile(optimizer=SGD(learning_rate=configurations['lr']), loss='binary_crossentropy')
print(model.summary())

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_embedding_user  │ (None, 1, 16)     │     96,640 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_embedding_item  │ (None, 1, 16)     │     59,296 │ item_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_6 (Flatten) │ (None, 16)        │          0 │ mlp_embedding_us… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_7 (Flatten) │ (None, 16)        │          0 │ mlp_embedding_it… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mf_embedding_user   │ (None, 1, 10)     │     60,400 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mf_embedding_item   │ (None, 1, 10)     │     37,060 │ item_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 32)        │          0 │ flatten_6[0][0],  │
│ (Concatenate)       │                   │            │ flatten_7[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_4 (Flatten) │ (None, 10)        │          0 │ mf_embedding_use… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_5 (Flatten) │ (None, 10)        │          0 │ mf_embedding_ite… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer1 (Dense)      │ (None, 16)        │        528 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_1          │ (None, 10)        │          0 │ flatten_4[0][0],  │
│ (Multiply)          │                   │            │ flatten_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer2 (Dense)      │ (None, 8)         │        136 │ layer1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 18)        │          0 │ multiply_1[0][0], │
│ (Concatenate)       │                   │            │ layer2[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ prediction (Dense)  │ (None, 1)         │         19 │ concatenate_2[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 254,079 (992.50 KB)

 Trainable params: 254,079 (992.50 KB)

 Non-trainable params: 0 (0.00 B)

None


In [26]:
topK = 10
evaluation_threads = 1 #mp.cpu_count()

In [27]:
# Init performance
t1 = time()
(hits, ndcgs) = evaluate_model(model, testRatings, testNegatives, topK, evaluation_threads)
hr, ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
#mf_embedding_norm = np.linalg.norm(model.get_layer('user_embedding').get_weights())+np.linalg.norm(model.get_layer('item_embedding').get_weights())
#p_norm = np.linalg.norm(model.get_layer('prediction').get_weights()[0])
print('Init: HR = %.4f, NDCG = %.4f\t [%.1f s]' % (hr, ndcg, time()-t1))

Init: HR = 0.0960, NDCG = 0.0438	 [361.8 s]


In [28]:
# Train model
print(f"Training NeuMF for {configurations['epochs']} epochs")
best_hr, best_ndcg, best_iter = hr, ndcg, -1
for epoch in range(1, configurations['epochs']+1):
    t1 = time()
    # Generate training instances
    user_input, item_input, labels = get_train_instances(train, configurations['num_neg'])

    # Training
    hist = model.fit([np.array(user_input), np.array(item_input)], #input
                      np.array(labels), # labels
                      batch_size=configurations['batch_size'], epochs=1, verbose=0, shuffle=True)
    t2 = time()

    # Evaluation
    if epoch % configurations['verbose'] == 0:
        (hits, ndcgs) = evaluate_model(model, testRatings, testNegatives, topK, evaluation_threads)
        hr, ndcg, loss = np.array(hits).mean(), np.array(ndcgs).mean(), hist.history['loss'][0]
        print('Iteration %d [%.1f s]: HR = %.4f, NDCG = %.4f, loss = %.4f [%.1f s]'
              % (epoch,  t2-t1, hr, ndcg, loss, time()-t2))
        if hr > best_hr:
            best_hr, best_ndcg, best_iter = hr, ndcg, epoch
            model_out_file = '%s_NeuMF_%d_%s.weights.h5' %(configurations['dataset'], configurations['num_factors'],configurations['layers'])
            if configurations['out'] > 0:
                model.save_weights(model_out_file, overwrite=True)
            print("End. Best Iteration %d:  HR = %.4f, NDCG = %.4f. " %(best_iter, best_hr, best_ndcg))
if configurations['out'] > 0:
    print("The best NeuMF model is saved to %s" %(model_out_file))

Training NeuMF for 10 epochs
Iteration 2 [39.0 s]: HR = 0.6215, NDCG = 0.3529, loss = 0.3395 [363.9 s]
End. Best Iteration 2:  HR = 0.6215, NDCG = 0.3529. 
Iteration 4 [38.8 s]: HR = 0.6535, NDCG = 0.3800, loss = 0.3150 [367.2 s]
End. Best Iteration 4:  HR = 0.6535, NDCG = 0.3800. 
Iteration 6 [38.8 s]: HR = 0.6677, NDCG = 0.3913, loss = 0.3066 [365.2 s]
End. Best Iteration 6:  HR = 0.6677, NDCG = 0.3913. 
Iteration 8 [38.7 s]: HR = 0.6664, NDCG = 0.3949, loss = 0.3021 [360.3 s]
Iteration 10 [38.5 s]: HR = 0.6719, NDCG = 0.3946, loss = 0.2979 [368.3 s]
End. Best Iteration 10:  HR = 0.6719, NDCG = 0.3946. 
The best NeuMF model is saved to ml-1m_NeuMF_10_[32, 16, 8].weights.h5
